In [1]:
import re
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# RAEX ТОП 100

In [4]:
years = list(range(2020, 2023 + 1))
raex_list = []

for y in years:
    raex_url = f"https://raex-rr.com/pro/education/russian_universities/top-100_universities/{y}/"
    raex_resp = requests.get(raex_url)

    if raex_resp.status_code == 200:
        raex_soup = BeautifulSoup(raex_resp.text)
        raex_trs = raex_soup.select('#rrp_table_wrapper > table > tbody.list > tr')

        for i, tr in enumerate(raex_trs):
            th = tr.findChildren('th', recursive=False)
            td = tr.findChildren('td', recursive=False)

            raex_list.append({
                'year': y,
                'rank': int(th[0].text),
                'title': re.sub(r'[\s\n]+', ' ', th[1].text).strip(),
                'previous': float(td[0].text) if td[0].text != '-' else np.NaN,
                'points': float(td[1].text),
                'quality': int(td[2].text),
                'graduates': int(td[3].text),
                'science': int(td[4].text)
            })
    else:
        print(f"{raex_url} return status code {raex_resp.status_code}")

    # Бережём источник данных
    time.sleep(1)

raex_df = pd.DataFrame(raex_list)
raex_df.to_csv(f"data/raex_{min(years)}-{max(years)}.csv", index=False)
raex_df.to_excel(f"data/raex_{min(years)}-{max(years)}.xlsx", index=False)
raex_df.head()

,year,rank,title,previous,points,quality,graduates,science
0,2020,1,Московский государственный университет имени М...,1.0,4.8419,1,1,1
1,2020,2,Московский физико-технический институт (национ...,2.0,4.7734,2,7,2
2,2020,3,Национальный исследовательский ядерный универс...,3.0,4.5535,5,5,4
3,2020,4,Санкт-Петербургский государственный университет,4.0,4.5394,3,12,7
4,2020,5,"Национальный исследовательский университет ""Вы...",5.0,4.4933,6,2,11


В отличие от предыдущих рейтингов, в данном рейтинге каждому университету присваивается конкретная позиция в рейтинге. Давайте посмотрим ТОП 10 лидеров по средней позиции в рейтинге за скачанные года.

In [6]:
raex_df[['title', 'rank']].groupby(by='title').mean().sort_values(by='rank').head(10)

,rank
title,
Московский государственный университет имени М.В. Ломоносова,1.000000
Московский физико-технический институт (национальный исследовательский университет),2.000000
Национальный исследовательский ядерный университет «МИФИ»,3.500000
Санкт-Петербургский государственный университет,3.500000
"Национальный исследовательский университет ""Высшая школа экономики""",5.250000
Московский государственный технический университет имени Н.Э. Баумана (национальный исследовательский университет),6.000000
МГИМО МИД России,6.750000
Санкт-Петербургский политехнический университет Петра Великого,8.250000
Национальный исследовательский Томский политехнический университет,8.666667


**Обратите внимание** на то, что рейтинговое агентство RAEX «богато» на рейтинги (полный список [тут](https://raex-rr.com/all_rankings/)). Для парсинга других рейтингов, механики будут +/- аналогичные.